## Importing the vertex amplitude as numpy array in Python

The EPRL vertex corresponds to the following object:

![alt text](pics/vertex_amplitude.png "Title")

in which $j_{12} = j_{13} = j_{14} = \dots = j_{45} = j$,. 

At fixed $j$, the amplitude can be interpreted as a 5-dim array with dimensions $(2j+1)^5$, where each entry corresponds to different
values for the intertwiners $i_1 \dots i_5$ (notice that $k_5$ in the pic is a typo) . 

In fact, there are $2j+1$ possible values for each intertwiner (from $0$ to $2j$).

In the folder are provided the vertices for $j = 0.5, 1, \dots 5$.

For example, we can load the 5-dimensional array corresponding to $j = 3$:

In [20]:
import numpy as np

j = 3;

vertex = np.load(f"../data/EPRL_vertices/python/vertex_j_{j}.0.npz");

In [21]:
vertex

array([[[[[ 3.26265100e-13, -4.98926708e-13,  5.20804485e-13, ...,
            4.70192631e-13, -5.30700527e-13,  8.93969417e-13],
          [-5.02359746e-13,  7.16869488e-13, -6.32016932e-13, ...,
           -1.42825485e-13, -2.11587323e-13,  1.10178600e-12],
          [ 5.26709695e-13, -6.35575451e-13,  3.25813148e-13, ...,
           -3.96256749e-13,  3.94252823e-13,  7.30665746e-13],
          ...,
          [ 4.64889848e-13, -1.43253745e-13, -3.98419546e-13, ...,
            4.38507786e-13,  2.83621243e-13,  9.63195000e-14],
          [-5.19781959e-13, -1.98785278e-13,  4.08485704e-13, ...,
            2.81562972e-13,  9.20210641e-14,  1.89670806e-14],
          [ 9.08658240e-13,  1.10759344e-12,  7.15762584e-13, ...,
            8.92406604e-14,  1.77222482e-14,  2.26129371e-15]],

         [[-4.76750071e-13,  7.40089009e-13, -7.89232949e-13, ...,
           -7.31500392e-13,  8.28888169e-13, -1.34343415e-12],
          [ 6.93751433e-13, -1.00751774e-12,  9.12228874e-13, ...,
      

### Correspondence between array elements and intertwiners

The amplitude $A \left( j, i_1 = 0, i_2 = 2, i_3 = 0, i_4 = 3, i_5 = 0 \right)$ correspons to the following element:

In [2]:
vertex[0,3,0,2,0]

-7.021548787502716e-13

etc.

In [47]:
from numpy.linalg import multi_dot

In [73]:
A = np.random.rand(5,5,5)
B = np.random.rand(5,5,5)
C = np.random.rand(5,5,5)

In [46]:
np.dot(a[:,0,0,0,0],b[:,0,0,0,0],b[:,0,0,0,0])

ValueError: output array is not acceptable (must have the right datatype, number of dimensions, and be a C-Array)

In [98]:
def naive_contraction(tensor, indices):

    result = 0.0

    for i in range(5):
       for j in range(5):
          for k in range(5):

             result += tensor[i,j,k] * tensor[indices[0], indices[1], i] * tensor[indices[2], indices[3], j] * tensor[indices[4], indices[5], k]

    return result         

In [101]:
tensor = np.random.rand(5,5,5)

indices = [0,1,4,3,0,1]

result = naive_contraction(tensor, indices)

print(result)

9.235668454775086


## Computing the dihedral angle (diagonal operator)

If we want to compute the dihedral angle on one node, the equation is:

$$
\langle O_1 \rangle = \frac{1}{Z} \sum_{i_1 \dots i_5} A^2 \left( j, i_1, i_2, i_3, i_4, i_5 \right) f \left( j, i_1 \right)
$$

where:

$$
f \left( j, i_1 \right) = \frac{i_1(i_1 + 1) - 2j(j+1)}{2j(j+1)}
$$

is the (diagonal) matrix element, and:

$$
Z = \sum_{i_1 \dots i_5} A^2 \left( j, i_1, i_2, i_3, i_4, i_5 \right)
$$

is the normalization factor. Of course we have:

$$
\sum_{i_1 \dots i_5} \equiv \sum_{i_1=0}^{2j} \sum_{i_2=0}^{2j} \sum_{i_3=0}^{2j} \sum_{i_4=0}^{2j} \sum_{i_5=0}^{2j}
$$

I don't remember if numpy stores arrays in row or column major order, therefore the computation below is probably not efficient

In [3]:
def boundary_angle(i,j):
    angle = (i*(i+1) - 2*j*(j+1)) / (2*j*(j+1));
    return angle

exp_value = 0.0
norm_factor = 0.0

for i1 in range(0,2*j+1):
    
    angle = boundary_angle(i1, j)
    
    for i2 in range(0,2*j+1): 
        for i3 in range(0,2*j+1): 
            for i4 in range(0,2*j+1): 
                for i5 in range(0,2*j+1): 
                    
                    A_squared = pow(vertex[i1,i2,i3,i4,i5],2)
                    
                    exp_value += A_squared*angle
                    norm_factor += A_squared    
                    
print(f"Expectation value is {exp_value/norm_factor}")                    

Expectation value is -0.33333333333333365


In [4]:
import tensorflow as tf


spin_j = j
vertex_amplitudes = tf.convert_to_tensor(
    vertex, dtype=tf.float64
)
squared_amplitudes = tf.math.square(vertex_amplitudes)
scale = tf.math.reduce_sum(squared_amplitudes)
# rewards = tf.cast(squared_amplitudes/scale, dtype=tf.float32)
rewards = squared_amplitudes/scale

2023-01-19 15:11:38.613782: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
rewards.shape

TensorShape([7, 7, 7, 7, 7])

In [8]:
inds_1d = tf.range(7)
inds = tf.meshgrid(*[inds_1d]*5, indexing='ij')

i1 = tf.cast(inds[0], dtype=tf.float64)

In [9]:
angle = (i1*(i1 + 1) - 2*spin_j*(spin_j + 1)) / (2*spin_j*(spin_j + 1))

In [10]:
tf.reduce_sum(rewards*angle)

<tf.Tensor: shape=(), dtype=float64, numpy=-0.33333333333333376>